<h1>Dog Breed Classification</h1>
<a id='Intro'></a>
<h1>Continued</h1>

This is a part two interactive notebook showcasing further modeling on the dog breed classification data. 
<br>
The part one notebook can be found: https://github.com/matthewjchin/dogbreedclassification/blob/main/dog_breed_class.ipynb

In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [9]:
# Import basic libraries
import pandas as pd
import numpy as np
import os

# Import plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns

np.random.seed(94)

In [10]:
os.getcwd()

'/kaggle/working'

In [11]:
os.chdir("/kaggle/input/dog-breed-identification")
os.getcwd()

'/kaggle/input/dog-breed-identification'

In [12]:
# The path to the files should not be like this all the time; everything done offline
# Actual project is found on Kaggle, but elected to run on own environment which best fit needs
labels = pd.read_csv("labels.csv")
labels.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [13]:
train_dir = '/kaggle/input/dog-breed-identification/train'

In [14]:
# Add entire folder path to image files and make it a column
labels['path'] = labels['id'].apply(lambda x : train_dir + "/"+ x + '.jpg')
labels.head()

,id,breed,path
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull,/kaggle/input/dog-breed-identification/train/0...
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo,/kaggle/input/dog-breed-identification/train/0...
2,001cdf01b096e06d78e9e5112d419397,pekinese,/kaggle/input/dog-breed-identification/train/0...
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick,/kaggle/input/dog-breed-identification/train/0...
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever,/kaggle/input/dog-breed-identification/train/0...


In [15]:
# Import TensorFlow, Keras, Scikit-learn libraries to be used

import tensorflow as tf # Use tf v2.12.0
import keras
from keras import layers, models, callbacks, optimizers
from keras.src.legacy.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array, load_img
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

tf.random.set_seed(104)

In [16]:
# Get the TensorFlow Version - we are using v2.12.0
print(tf.__version__, keras.__version__)

2.18.0 3.8.0


In [17]:
# Set up constants for model
RANDOM_STATE = 50
IMG_SIZE = 256
DOG_BREEDS = len(labels['breed'].unique())
LEARNING_RATE = 0.005
BATCH_SIZE = 32

# Epochs value may vary based on number of tests
EPOCHS = 5

In [18]:
# Check if GPU is available - only on Colab or Kaggle

gpus = tf.config.list_physical_devices('GPU')

if gpus:
    print("GPU being used for training")
    for gpu in gpus:
        print(f"GPU Name: {gpu.name}")
else: # Use CPU
    BATCH_SIZE=16
    EPOCHS = 5
    print("Using CPU")


GPU being used for training
GPU Name: /physical_device:GPU:0
GPU Name: /physical_device:GPU:1


In [19]:
# Create a label encoding
encoder = LabelEncoder()
labels['breed_id']= encoder.fit_transform(labels['breed'])
labels.head()


,id,breed,path,breed_id
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull,/kaggle/input/dog-breed-identification/train/0...,19
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo,/kaggle/input/dog-breed-identification/train/0...,37
2,001cdf01b096e06d78e9e5112d419397,pekinese,/kaggle/input/dog-breed-identification/train/0...,85
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick,/kaggle/input/dog-breed-identification/train/0...,15
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever,/kaggle/input/dog-breed-identification/train/0...,49


In [20]:
'''Set up variables x, y to load images from train_dir into array/matrix form (seen in last cell)
Adjust size as appropriate and put everything in list form
Create dataset y of dummy variables of dog breeds and then get shapes

x - type numpy array
y - type dataframe
'''


x= np.array([img_to_array(load_img(image, target_size=(IMG_SIZE, IMG_SIZE)))
             for image in labels['path'].values.tolist()])
y = pd.get_dummies(labels['breed'])
print(x.shape, y.shape)

(10222, 256, 256, 3) (10222, 120)


In [22]:
# Do the train-test split
# Implement the training and validation sets for x, y so that train-test size is 80%-20%
 
x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.2)

print(x_train.shape,y_train.shape)
print(x_valid.shape,y_valid.shape)

(8177, 256, 256, 3) (8177, 120)
(2045, 256, 256, 3) (2045, 120)


In [23]:
print(type(x_train), type(x_valid), type(y_train), type(y_valid))
print(x_train.dtype, x_valid.dtype)

<class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'>
float32 float32


In [1]:
# See how this is 4-dimensional, will have to reshape into
# 2-dimensional structure before making it a dataframe

print(x_train.shape, x_valid.shape)

NameError: name 'x_train' is not defined

In [27]:
# Reshape the x_train and x_valid arrays
x_train = x_train.reshape(-1)
x_valid = x_valid.reshape(-1)

In [28]:
print(x_train, x_valid)

[171. 205. 172. ...  96.  97.  99.] [ 95. 115. 106. ...  27.  38.  22.]


In [ ]:
# Change types of values from float to int
x_train = x_train.astype(int)
x_valid = x_valid.astype(int)
print(x_train.dtype, x_valid.dtype)

<a id='Different'></a>
## <h1>Trying a Different Model</h1>

Try another model to see if we can get better predictions.<br>
Use VGG16 deep CNN for image classification, where there are 16 layers of artificial neurons.<br>

First, perform some image augmentation using ImageDataGenerator on the train data.<br>
Also, set up a data generator for validation data.<br>

We will use VGG16 as the architecture to try and get better predictions.

In [21]:

train_data_gen = ImageDataGenerator(
    rescale=1./255., rotation_range=40,
    width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2,
    zoom_range=0.2, horizontal_flip=True
)

valid__data_gen = ImageDataGenerator(rescale=1./ 255.)